In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
dotenv.load_dotenv()

/opt/anaconda3/envs/langchain_llm-env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "service_Account_key.json"
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [3]:
pc = Pinecone(api_key=pinecone_api_key)

embedding_client = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

## Try out embeddings

In [4]:
embeddings = embedding_client.embed_query("hello there")
print(embeddings)

[0.031404782086610794, -0.0006279340013861656, -0.026396891102194786, -0.03786284849047661, 0.004164970014244318, 0.04849613830447197, 0.09854729473590851, 0.023019876331090927, 0.027665389701724052, 0.014419440180063248, -0.08015839010477066, 0.017120784148573875, 0.024943867698311806, -0.015763340517878532, -0.049686748534440994, -0.01851409673690796, 0.0007687789620831609, 0.017062129452824593, -0.05472962558269501, -0.02884751930832863, -0.008814419619739056, 0.0007315335096791387, 0.004280418623238802, -0.021152537316083908, -0.013978148810565472, 0.047206662595272064, -0.0002639206941239536, -0.01861683651804924, 0.023759273812174797, -0.014043321833014488, -0.039888687431812286, 0.052715737372636795, -0.011856728233397007, -0.025385349988937378, 0.010475334711372852, 0.003867280902341008, -0.026699211448431015, 0.02142431028187275, 0.02272053062915802, -0.08938819915056229, -0.029820052906870842, 0.0717545598745346, 0.00029729606467299163, -0.01300576701760292, -0.00323868938721

In [5]:
len(embeddings)

768

## Wrangle dataset

In [6]:
df=pd.read_json('Products_Info/products.jsonl',lines=True)

In [7]:
df.head(3)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,Latte.jpg


In [8]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [9]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [10]:
texts = df['text'].tolist()
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Espre

In [11]:
with open('Products_Info/About_Us_Info.txt') as f:
    DreamBucksCoffee_about_section = f.read()
    
DreamBucksCoffee_about_section = "Coffee shop DreamBucks Coffee about section: " + DreamBucksCoffee_about_section
texts.append(DreamBucksCoffee_about_section)

In [12]:
with open('Products_Info/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

In [13]:
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Espre

## Generate Embeddings

In [14]:
all_data_embeddings = [embedding_client.embed_query(x) for x in texts]

In [15]:

print(len(all_data_embeddings))
print(len(all_data_embeddings[0]))
print(all_data_embeddings)

20
768
[[0.005776355974376202, -0.008747355081140995, -0.022011248394846916, -0.0006296541541814804, 0.026473481208086014, 0.02155717834830284, 0.008611353114247322, 0.003957613371312618, 0.016584839671850204, 0.031983837485313416, -0.02155366726219654, 0.11952951550483704, 0.022053061053156853, 0.04251808300614357, 0.06001519784331322, -0.08574051409959793, 0.006636194884777069, 0.11056146025657654, -0.0837043970823288, 0.0015353948110714555, 0.01129004918038845, 0.028805183246731758, 0.006161203607916832, -0.013588416390120983, -0.034523118287324905, -0.014129161834716797, -0.05478432774543762, 0.03253808617591858, 0.012460008263587952, -0.010107406415045261, -0.015650726854801178, -0.007404588162899017, 0.005189827177673578, -0.025629498064517975, -0.022394724190235138, 0.008241333067417145, -0.04860944673418999, 0.002223910763859749, 0.0017600501887500286, -0.06045633554458618, -0.06588492542505264, -0.03564303368330002, -0.030917204916477203, 0.06010380759835243, 0.016440819948911

## Push data to database

##### In Vector databases, we don't need to manually write cosine similarities and get all the data. The Vector database does it by itself efficiently.

##### Create a Pinecone index

In [16]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=768, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

#### Making vectors with metadata to be inserted into the index of pinecone

In [17]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

vectors = []
for text, e in zip(texts, all_data_embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e,
        "metadata": {'text': text}
    })

In [18]:
print(len(vectors))
print(vectors[0])


20
{'id': 'Cappuccino', 'values': [0.005776355974376202, -0.008747355081140995, -0.022011248394846916, -0.0006296541541814804, 0.026473481208086014, 0.02155717834830284, 0.008611353114247322, 0.003957613371312618, 0.016584839671850204, 0.031983837485313416, -0.02155366726219654, 0.11952951550483704, 0.022053061053156853, 0.04251808300614357, 0.06001519784331322, -0.08574051409959793, 0.006636194884777069, 0.11056146025657654, -0.0837043970823288, 0.0015353948110714555, 0.01129004918038845, 0.028805183246731758, 0.006161203607916832, -0.013588416390120983, -0.034523118287324905, -0.014129161834716797, -0.05478432774543762, 0.03253808617591858, 0.012460008263587952, -0.010107406415045261, -0.015650726854801178, -0.007404588162899017, 0.005189827177673578, -0.025629498064517975, -0.022394724190235138, 0.008241333067417145, -0.04860944673418999, 0.002223910763859749, 0.0017600501887500286, -0.06045633554458618, -0.06588492542505264, -0.03564303368330002, -0.030917204916477203, 0.0601038075

In [19]:
index = pc.Index(index_name)
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

## Get Closest documents

In [20]:
embedding = embedding_client.embed_query("Is Cappuccino lactose-free?")
embedding

[0.00010746183397714049,
 -0.004896368365734816,
 -0.04651745408773422,
 -0.04684653878211975,
 -0.010465292260050774,
 -0.007662342395633459,
 0.01763838157057762,
 0.0026871764566749334,
 0.013535907492041588,
 0.07266981154680252,
 -0.00021518391440622509,
 0.037441954016685486,
 -0.014972453005611897,
 0.05634040758013725,
 0.015360665507614613,
 -0.034944742918014526,
 -0.0006632859585806727,
 0.0631936639547348,
 -0.042116980999708176,
 0.023545298725366592,
 0.04442037269473076,
 0.041460342705249786,
 0.01569085568189621,
 0.004896469879895449,
 -0.014102141372859478,
 0.022471748292446136,
 -0.027407227084040642,
 0.007178433705121279,
 -0.02860945649445057,
 -0.06737296283245087,
 -0.03851264342665672,
 0.00449966499581933,
 0.04461768642067909,
 -0.003395672654733062,
 -0.005967258010059595,
 -0.028217988088726997,
 -0.03137029707431793,
 0.10526023805141449,
 -0.01286227535456419,
 -0.07947859168052673,
 -0.03663647174835205,
 -0.014025969430804253,
 0.0097523033618927,
 0.

In [23]:
results = index.query(
    namespace="ns1",
    vector=embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.629178286,
              'values': []},
             {'id': 'Latte',
              'metadata': {'text': 'Latte : Smooth and creamy, our latte '
                                   'combines rich espresso with velvety '
                 